In [1]:
import cv2
import os
import numpy as np
from tqdm import tqdm
import glob
import torch
import pandas as pd
import matplotlib.pyplot as plt
import gc
import torchvision

from os.path import join as join_path

from PIL import Image
from torchvision import transforms

In [2]:
PATH_TO_FOLDER = '/home/jovyan/work/'
path = join_path(PATH_TO_FOLDER, 'run_attacks')
SAVE_PATH = join_path(PATH_TO_FOLDER, 'run_attacks', 'tmp_results')

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
DEVICE = device
device

device(type='cuda', index=0)

# Example

In [4]:
from torchvision import transforms
from torch.autograd import Variable

In [5]:
def adv(ref, eps, alpha, iters):
    image = transforms.ToTensor()(ref)
    image = image.unsqueeze_(0)
    image = image.to(DEVICE)
    p = torch.zeros_like(image).to(DEVICE)
    p = Variable(p, requires_grad=True)
    model = Linearity(DEVICE)

    for i in range(iters):
        res = image + p
        res.data.clamp_(0., 1.)
        score = model(res)
        print("linearity score:", score.item())
        loss = 1 - score / 100
        loss.backward() 
        g = p.grad
        g = torch.sign(g)
        p.data -= alpha * g
        p.data.clamp_(-eps, +eps)
        p.grad.zero_()

    res_image = (image).data.clamp_(min=0, max=1)
    res_img = (res_image.squeeze().data.cpu().numpy().transpose(1, 2, 0) * 255).astype('uint8')
    return res_img

# images = sorted(os.listdir('test_ims'))
# im = cv2.imread('test_ims/'+images[0])
# im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
# eps = 10/255
# alpha = 1/255
# iters = 10
# im = adv(im, eps, alpha, iters)
# im = cv2.cvtColor(im, cv2.COLOR_RGB2BGR)
# os.makedirs('res_fgsm', exist_ok=True)
# cv2.imwrite('res_fgsm/'+images[0], im)

# Run IQA models

In [12]:
# _type_folder = 'TID2013/distorted_images' # val2017 train2017 test2017

_type_folder = 'KADID10K/kadid10k/images'
image_type_folder = 'kadid10k'

all_images = sorted(glob.glob(join_path(PATH_TO_FOLDER, 'data', _type_folder, '*')))
# image_type_folder = f'mscoco_{_type_folder}'
# image_type_folder = _type_folder
# image_type_folder = 'tid2013'

print(len(all_images))

10206


In [7]:
_type_folder = 'train2017'

# _type_folder = 'KADID10K/kadid10k/images'
# image_type_folder = 'kadid10k'

all_images = [join_path(PATH_TO_FOLDER, 'data', _type_folder, '000000009898.jpg'), join_path(PATH_TO_FOLDER, 'data', _type_folder, '000000333018.jpg')]
image_type_folder = f'mscoco_{_type_folder}'
# image_type_folder = _type_folder
# image_type_folder = 'tid2013'

In [13]:
# df = pd.read_csv(join_path(PATH_TO_FOLDER, 'data', 'KADID10K', 'images_HackPredictModelV31_padding.csv'))
# df['distortion_type'] = df['image_name'].apply(lambda x: int(x.split('_')[1]) if len(x) != 7 else 0)
# ims = list(set(df[df['distortion_type'] == 6]['image_name'])) + list(set(df[df['distortion_type'] == 22]['image_name']))
# all_images = [join_path(PATH_TO_FOLDER, 'data', 'KADID10K', 'kadid10k', 'images', i) for i in ims]
# all_images = sorted(all_images)

# image_type_folder = 'kadid10k_only_6_22'

# len(all_images)

## Linearity

In [8]:
from argparse import ArgumentParser
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms.functional import resize, to_tensor, normalize
from PIL import Image
# import h5py

In [9]:
os.chdir(join_path(PATH_TO_FOLDER, 'metrics', 'Linearity'))
from IQAmodel import IQAModel

In [10]:
model = IQAModel().to(device)
model_path = '/home/jovyan/storage/NR-metric-models/p1q2.pth'
checkpoint = torch.load(model_path)
k = checkpoint['k']
b = checkpoint['b']
model.load_state_dict(checkpoint['model'])
model.eval();

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt101_32X8D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt101_32X8D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
eps = 3 / 255
alpha = 1 / 255
iters = 2
fname = f'ifgsm_alpha{int(alpha * 255)}_eps{int(eps * 255)}_i{iters}_lin_{image_type_folder}.csv'
fname

'ifgsm_alpha1_eps3_i2_lin_mscoco_train2017.csv'

In [12]:
res = []
gains = []
for image_file in tqdm(all_images[:1]):
    image = Image.open(image_file).convert('RGB')
    
    image = resize(image, (498, 664))
    image = to_tensor(image).to(device)
    image = image.unsqueeze(0)
    p = torch.zeros_like(image).to(DEVICE)
    p = Variable(p, requires_grad=True)
    before_score = 0
    for i in range(iters):
        image_for_attack = image + p
        image_for_attack.data.clamp_(0., 1.)
        image_for_attack = normalize(image_for_attack, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        score = model(image_for_attack)[-1]
        score = score * k[-1] + b[-1]
        if i == 0:
            before_score = score.cpu().detach().item()
        loss = 1 - score / 100
        loss.backward() 
        g = p.grad
        g = torch.sign(g)
        p.data -= alpha * g
        p.data.clamp_(-eps, +eps)
        p.grad.zero_()
        
    after_score = score.cpu().detach().item()
    
    gains.append([before_score, after_score])
    res.append([image_file.split('/')[-1], before_score, after_score])
    
    
gains = np.array(gains)

df = pd.DataFrame(res, columns=['image_name', 'before_score', 'after_score'])
df['attack_type'] = f'ifgsm_alpha{int(alpha * 255)}_eps{int(eps * 255)}_i{iters}'
df['metric'] = 'lin'

x = (gains[:, 1] - gains[:, 0]) # .mean()
x.mean()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.16s/it]


25.21416473388672

In [13]:
Image.fromarray(image_after_attack)#.save('000000333018_attack_fgsm_30.jpg')

NameError: name 'image_after_attack' is not defined

In [ ]:
df.to_csv(join_path(SAVE_PATH, fname))

## Paq-2-Piq

In [ ]:
os.environ['my_device'] = str(device)

os.chdir(join_path(PATH_TO_FOLDER, 'metrics', 'Paq-2-Piq'))
from paq2piq_standalone import InferenceModel, RoIPoolModel

In [ ]:
# model = InferenceModel(RoIPoolModel().to(device), 'D:\\me\\iqa_aimasters\\metrics\\Paq-2-Piq\\RoIPoolModel.pth')

In [ ]:
model = RoIPoolModel()
model_path = join_path(PATH_TO_FOLDER, 'metrics', 'Paq-2-Piq', 'RoIPoolModel.pth')
model_state = torch.load(model_path, map_location=lambda storage, loc: storage)
model.load_state_dict(model_state["model"])
model.to(device)
model.eval();

In [ ]:
p2p_transform = transforms.ToTensor()

In [ ]:
eps = 5 / 255

alpha = 2 / 255
iters = 3

In [ ]:
res = []
gains = []
for image_file in tqdm(all_images):
    image = cv2.imread(image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = p2p_transform(image).to(device)
    image = image.unsqueeze(0)
    model.input_block_rois((20, 20), [image.shape[-2], image.shape[-1]], device=device)
    
    p = torch.zeros_like(image).to(DEVICE)
    p = Variable(p, requires_grad=True)
    before_score = 0
    for i in range(iters):
        image_for_attack = image + p
        image_for_attack.data.clamp_(0., 1.)
        score = model(image_for_attack)[0][0]
        if i == 0:
            before_score = score.cpu().detach().item()
        # print("score:", score.item())
        loss = 1 - score / 100
        loss.backward() 
        g = p.grad
        g = torch.sign(g)
        p.data -= alpha * g
        p.data.clamp_(-eps, +eps)
        p.grad.zero_()
        
    after_score = score.cpu().detach().item()
    
    gains.append([before_score, after_score])
    res.append([image_file.split('\\')[-1], before_score, after_score])
gains = np.array(gains)

df = pd.DataFrame(res, columns=['image_name', 'before_score', 'after_score'])
df['attack_type'] = f'ifgsm_alpha{int(alpha * 255)}_255_eps{int(eps * 255)}_255_i{iters}'
df['metric'] = 'p2p'

x = (gains[:, 1] - gains[:, 0]) # .mean()
x.mean()

In [ ]:
fname = f'ifgsm_alpha{int(alpha * 255)}_eps{int(eps * 255)}_i{iters}_p2p_{image_type_folder}.csv'
df.to_csv(join_path(SAVE_PATH, fname))

## Nima

In [ ]:
# os.chdir('D:\\me\\iqa_aimasters\\metrics\\Nima\\nima')

# from inference.inference_model import InferenceModel

# path_to_model_weight = "D:\\me\\iqa_aimasters\\metrics\\Nima\\pretrain-model.pth" 
# model = InferenceModel(path_to_model=path_to_model_weight)

In [ ]:
# eps = 5 / 255

# alpha = 2 / 255
# iters = 2

In [ ]:
# res = []
# gains = []
# for image_file in tqdm(all_images):
#     im = Image.open(image_file).convert('RGB')
#     image = model.transform(im)
#     image = image.to(device)
#     image = image.unsqueeze(0)
    
#     p = torch.zeros_like(image).to(DEVICE)
#     p = Variable(p, requires_grad=True)
#     before_score = 0
#     for i in range(iters):
#         image_for_attack = image + p
#         image_for_attack.data.clamp_(0., 1.)
#         prob = model.model(image_for_attack)[0]
#         score = (model.const_nima * prob).sum()
#         if i == 0:
#             before_score = score.cpu().detach().item()
#         # print("score:", score.item())
#         loss = 1 - score / 100
#         loss.backward() 
#         g = p.grad
#         g = torch.sign(g)
#         p.data -= alpha * g
#         p.data.clamp_(-eps, +eps)
#         p.grad.zero_()
        
#     after_score = score.cpu().detach().item()
    
#     gains.append([before_score, after_score])
#     res.append([image_file.split('\\')[-1], before_score, after_score])
# gains = np.array(gains)

# df = pd.DataFrame(res, columns=['image_name', 'before_score', 'after_score'])
# df['attack_type'] = f'ifgsm_alpha{int(alpha * 255)}_255_eps{int(eps * 255)}_255_i{iters}'
# df['metric'] = 'nima'

# x = (gains[:, 1] - gains[:, 0]) # .mean()
# x.mean()

In [ ]:
# fname = f'ifgsm_alpha{int(alpha * 255)}_255_eps{int(eps * 255)}_255_i{iters}_nima_mscoco_val2017'
# df.to_csv(join_path(PATH_TO_FOLDER, 'results', f'{fname}.csv'))

## SPAQ

In [ ]:
os.chdir(join_path(PATH_TO_FOLDER, 'metrics', 'SPAQ'))

from Prepare_image import Image_load


class Baseline(torch.nn.Module):
    def __init__(self):
        super(Baseline, self).__init__()
        self.backbone = torchvision.models.resnet50(pretrained=False)
        fc_feature = self.backbone.fc.in_features
        self.backbone.fc = torch.nn.Linear(fc_feature, 1, bias=True)

    def forward(self, x):
        result = self.backbone(x)
        return result

model = Baseline()
model_path = '/home/jovyan/storage/NR-metric-models/BL_release.pt'
checkpoint = torch.load(model_path, device)
model.load_state_dict(checkpoint['state_dict'])
model.to(device);
model.eval();

prepare_image = Image_load(size=512, stride=224)

to_tensor = transforms.ToTensor()

In [ ]:
eps = 3 / 255

alpha = 1 / 255
iters = 2
fname = f'ifgsm_alpha{int(alpha * 255)}_eps{int(eps * 255)}_i{iters}_spaq_{image_type_folder}.csv'
print(fname)

In [ ]:
res = []
gains = []
for image_file in tqdm(all_images):
    try:
        # image = prepare_image(Image.open(image_file).convert('RGB'), attack=False).to(device)
        # before_score = model(image).mean().cpu().detach().cpu().item()
        image = prepare_image(Image.open(image_file).convert('RGB'), attack=True).to(device)
    except:
        res.append([image_file.split('/')[-1], -1, -1])
        continue

    p = torch.zeros_like(image).to(DEVICE)
    p = Variable(p, requires_grad=True)
    # before_score = 0
    before_score = -1
    for i in range(iters):
        image_for_attack = image + p
        image_for_attack.data.clamp_(0., 1.)
        score = model(image_for_attack).mean()
        # if i == 0:
        #     before_score = score.cpu().detach().item()
        loss = 1 - score / 100
        loss.backward() 
        g = p.grad
        g = torch.sign(g)
        p.data -= alpha * g
        p.data.clamp_(-eps, +eps)
        p.grad.zero_()
        
    # after_score = score.cpu().detach().item()
    
    try:
        image_for_attack = (image_for_attack.squeeze().data.cpu().numpy().transpose(1, 2, 0) * 255).astype('uint8')
        img = prepare_image(Image.fromarray(image_for_attack).convert('RGB'), attack=False).to(device)
        after_score = model(img).mean().cpu().detach().item()
    except:
        after_score = -1
    
    gains.append([before_score, after_score])
    res.append([image_file.split('/')[-1], before_score, after_score])
gains = np.array(gains)

df = pd.DataFrame(res, columns=['image_name', 'before_score', 'after_score'])
df['attack_type'] = f'ifgsm_alpha{int(alpha * 255)}_eps{int(eps * 255)}_i{iters}'
df['metric'] = 'spaq'

print("gain diff: ", (gains[:, 1] - gains[:, 0]).mean())
print("gain percent:", ((gains[:, 1] - gains[:, 0]) / gains[:, 0]).mean())

In [ ]:
df.to_csv(join_path(SAVE_PATH, fname))

## KonCept512

In [ ]:
import torch.nn as nn
import argparse
import os
from torchvision.transforms import ToTensor, Resize, Normalize
from torchvision import transforms

In [ ]:
os.chdir(join_path(PATH_TO_FOLDER, 'metrics', 'KonCept512'))

from inceptionresnetv2 import inceptionresnetv2

In [ ]:
class model_qa(nn.Module):
    def __init__(self,num_classes,**kwargs):
        super(model_qa,self).__init__()
        base_model = inceptionresnetv2(num_classes=1000, pretrained='imagenet')
        self.base = nn.Sequential(*list(base_model.children())[:-1])
        self.fc = nn.Sequential(
            nn.Linear(1536, 2048),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(2048),
            nn.Dropout(p=0.25),
            nn.Linear(2048, 1024),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(1024),
            nn.Dropout(p=0.25),
            nn.Linear(1024, 256),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(256),         
            nn.Dropout(p=0.5),
            nn.Linear(256, num_classes),
        )

    def forward(self,x):
        x = self.base(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x    

In [ ]:
KonCept512 = model_qa(num_classes=1)
KonCept512.load_state_dict(torch.load('/home/jovyan/storage/NR-metric-models/KonCept512.pth'))
KonCept512.eval().to(device);

In [ ]:
koncept_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((384, 512))
    ])
koncept_normalize = transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
eps = 2 / 255

alpha = 3 / 255
iters = 3

In [ ]:
res = []
gains = []
for image_file in tqdm(all_images):
    image = cv2.cvtColor(cv2.imread(image_file), cv2.COLOR_BGR2RGB)
    image = koncept_transform(image).to(device)
    image = koncept_normalize(image).unsqueeze(0)
    p = torch.zeros_like(image).to(DEVICE)
    p = Variable(p, requires_grad=True)
    before_score = 0
    for i in range(iters):
        image_for_attack = image + p
        # print(image_for_attack.max(), image_for_attack.min())
        image_for_attack.data.clamp_(-1., 1.)
        # image_for_attack = koncept_normalize(image_for_attack) # .unsqueeze(0)
        score = KonCept512(image_for_attack)[0][0]
        if i == 0:
            before_score = score.cpu().detach().item()
        # print("score:", score.item())
        loss = 1 - score / 100
        loss.backward() 
        g = p.grad
        g = torch.sign(g)
        p.data -= alpha * g
        p.data.clamp_(-eps, +eps)
        p.grad.zero_()
        
    after_score = score.cpu().detach().item()
    
    gains.append([before_score, after_score])
    res.append([image_file.split('\\')[-1], before_score, after_score])
gains = np.array(gains)

df = pd.DataFrame(res, columns=['image_name', 'before_score', 'after_score'])
df['attack_type'] = f'ifgsm_alpha{int(alpha * 255)}_255_eps{int(eps * 255)}_255_i{iters}'
df['metric'] = 'koncept'

x = (gains[:, 1] - gains[:, 0]) # .mean()
x.mean()

In [ ]:
fname = f'ifgsm_alpha{int(alpha * 255)}_eps{int(eps * 255)}_i{iters}_koncept_{image_type_folder}.csv'
df.to_csv(join_path(SAVE_PATH, fname))

## MDTVSFA

In [ ]:
import torch
import random
# from torchvision import transforms
import skvideo.io
from PIL import Image
import numpy as np

torch.manual_seed(0)
random.seed(0)
np.random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.utils.backcompat.broadcast_warning.enabled = True

In [ ]:
os.chdir(join_path(PATH_TO_FOLDER, 'metrics', 'MDTVSFA'))

from VQAmodel import VQAModel
from CNNfeatures import CNNModel

In [ ]:
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
to_tensor = transforms.ToTensor()

In [ ]:
model = VQAModel().to(device)
model.load_state_dict(torch.load('/home/jovyan/storage/NR-metric-models/MDTVSFA.pt'))

In [ ]:
extractor = CNNModel(model='ResNet-50').to(device);

In [ ]:
model.train();
extractor.eval();

In [ ]:
eps = 2 / 255

alpha = 2 / 255
iters = 3

In [ ]:
res = []
gains = []
for image_file in tqdm(all_images): 
    image = Image.open(image_file).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    p = torch.zeros_like(image).to(device)
    p = Variable(p, requires_grad=True)
    before_score = 0
    for i in range(iters):
        image_for_attack = image + p
        features_mean, features_std = extractor(image_for_attack)
        features = torch.cat((features_mean, features_std), 1).squeeze()
        features = torch.unsqueeze(features, 0)
        input_length = features.shape[1] * torch.ones(1, 1, dtype=torch.long)
        score = model((features, input_length))[0]
        image_for_attack.data.clamp_(-1., 1.)        
        if i == 0:
            before_score = score.detach().cpu().item()
        loss = 1 - score / 100
        loss.backward()
        g = p.grad
        g = torch.sign(g)
        p.data -= alpha * g
        p.data.clamp_(-eps, +eps)
        p.grad.zero_()
        
    after_score = score.cpu().detach().item()
    
    gains.append([before_score, after_score])
    res.append([image_file.split('/')[-1], before_score, after_score])
    
gains = np.array(gains)

df = pd.DataFrame(res, columns=['image_name', 'before_score', 'after_score'])
df['attack_type'] = f'ifgsm_alpha{int(alpha * 255)}_eps{int(eps * 255)}_i{iters}'
df['metric'] = 'mdtvsfa'

In [ ]:
print("gain diff: ", (gains[:, 1] - gains[:, 0]).mean())
print("gain percent:", ((gains[:, 1] - gains[:, 0]) / gains[:, 0]).mean())

In [ ]:
fname = f'ifgsm_alpha{int(alpha * 255)}_eps{int(eps * 255)}_i{iters}_mdtvsfa_{image_type_folder}.csv'
df.to_csv(join_path(SAVE_PATH, fname))

In [ ]:
image_type_folder

In [ ]:
df